In [2]:
import requests

In [ ]:
region='test'
if region == 'test':
    host = 'vnatest1vt'
    port = '8083'
elif region == 'prod':
    host = 'vnatest1vt'
    port = '8083'
else:
    print("Unsupported region")        


In [ ]:
studies=None

query_str = '&'.join(['requestType=WADO','contenttype=image/jpeg'])
if studies is not None:
    query_str = '/studies/'+studies
url = ''.join(['http://', host, ':', port, '/AcuoREST', query_str])

r = requests.get("http://vnatest1vt:8083/AcuoREST/dicomrs/retrieve/studies/1.2.392.200036.9116.2.5.1.48.1216060901.1336345628.42/metadata", auth=(user, pw))
    
    return r.json()
"http://vnatest1vt:8083/AcuoREST/dicomrs/retrieve/studies/1.2.392.200036.9116.2.5.1.48.1216060901.1336345628.42/metadata"

In [6]:
user='wangkcc4'
pw=''
r = requests.get("http://vnatest1vt:8083/AcuoREST/dicomrs/retrieve/studies/1.2.392.200036.9116.2.5.1.48.1216060901.1336345628.42/series/1.2.392.200036.9116.2.1216060901.1336346550.3.1004100001.7/instances/1.2.392.200036.9116.2.1216060901.1336346841.684518.7.1277", auth=(user, pw))
r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)